# TEORÍA - SESIÓN 4: Testing e Integración con IA Generativa

**Duración:** 180 minutos (3 horas)

**Objetivos:**
- Implementar tests automatizados con pytest y TestClient
- Mockear dependencias y autenticación en tests
- Integrar APIs de IA generativa en FastAPI
- Controlar tokens, costes y rate limiting
- Manejar errores y timeouts en llamadas a IA

## CONFIGURACIÓN DEL ENTORNO

In [ ]:
# Instalación de dependencias
!pip install fastapi==0.115.0 uvicorn[standard]==0.32.0 pytest==8.0.0 httpx==0.27.0 openai==1.12.0 python-dotenv==1.0.0 -q
print("✅ Dependencias instaladas")

In [ ]:
# Imports necesarios
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.testclient import TestClient
from pydantic import BaseModel
from typing import Annotated, Optional
import pytest
import os
from datetime import datetime
import time

print("✅ Imports completados")

---

# 🧪 BLOQUE 1: TESTING (90 minutos)

## 1. INTRODUCCIÓN AL TESTING (10 min)

### ¿Por qué testear APIs?

Los tests automatizados son esenciales en producción:

1. **Detectar bugs antes de desplegar** → Evitar errores en producción
2. **Documentación viva** → Los tests muestran cómo usar la API
3. **Refactorización segura** → Cambiar código sin miedo a romper funcionalidad
4. **CI/CD** → Integración continua con validación automática

### TestClient de FastAPI

FastAPI incluye `TestClient` basado en `httpx` para hacer requests sin levantar un servidor real.

**Ventajas:**
- No necesitas `uvicorn` corriendo
- Tests rápidos (milisegundos)
- Aislamiento total (cada test limpio)

### Estructura básica con pytest

```python
# test_api.py
from fastapi.testclient import TestClient

def test_nombre_descriptivo():
    # Arrange (preparar)
    client = TestClient(app)
    
    # Act (actuar)
    response = client.get("/ruta")
    
    # Assert (verificar)
    assert response.status_code == 200
```

### Ejemplo básico: API simple con test

In [ ]:
# API simple
app = FastAPI()

@app.get("/")
def root():
    return {"mensaje": "API funcionando"}

@app.get("/saludo/{nombre}")
def saludar(nombre: str):
    return {"saludo": f"Hola {nombre}"}

# Test con TestClient
client = TestClient(app)

# Test 1: Endpoint raíz
response = client.get("/")
assert response.status_code == 200
assert response.json() == {"mensaje": "API funcionando"}
print("✅ Test 1 pasado: Endpoint raíz funciona")

# Test 2: Endpoint con parámetro
response = client.get("/saludo/Ana")
assert response.status_code == 200
assert response.json()["saludo"] == "Hola Ana"
print("✅ Test 2 pasado: Saludo personalizado funciona")

---

## 2. TESTS BÁSICOS (10 min + micro-reto)

### Status codes

Los tests más comunes verifican los códigos HTTP correctos:

- **200 OK** → GET exitoso
- **201 Created** → POST crea recurso
- **400 Bad Request** → Datos inválidos
- **401 Unauthorized** → Sin autenticación
- **404 Not Found** → Recurso no existe
- **422 Unprocessable Entity** → Validación Pydantic falla

In [ ]:
# API con validación Pydantic
app_validacion = FastAPI()

class Item(BaseModel):
    nombre: str
    precio: float
    cantidad: int = 1

@app_validacion.post("/items", status_code=201)
def crear_item(item: Item):
    return {"item_creado": item.dict()}

# Tests de status codes
client_val = TestClient(app_validacion)

# Test 1: POST exitoso retorna 201
response = client_val.post("/items", json={"nombre": "Laptop", "precio": 999.99})
assert response.status_code == 201
print("✅ Test: POST exitoso retorna 201")

# Test 2: POST con datos inválidos retorna 422
response = client_val.post("/items", json={"nombre": "Laptop"})  # Falta precio (obligatorio)
assert response.status_code == 422
print("✅ Test: Validación Pydantic retorna 422")

# Test 3: Verificar estructura de error 422
error_detail = response.json()["detail"]
assert len(error_detail) > 0  # Hay al menos un error
assert error_detail[0]["type"] == "missing"  # Campo faltante
print("✅ Test: Error 422 tiene estructura correcta")

### Validación de estructura de respuesta

Además del status code, debemos verificar que el JSON retornado tenga la estructura esperada.

In [ ]:
# Test completo de estructura
response = client_val.post("/items", json={
    "nombre": "Mouse",
    "precio": 25.50,
    "cantidad": 3
})

assert response.status_code == 201
data = response.json()

# Verificar que existe la clave "item_creado"
assert "item_creado" in data

# Verificar estructura interna
item = data["item_creado"]
assert item["nombre"] == "Mouse"
assert item["precio"] == 25.50
assert item["cantidad"] == 3

print("✅ Test: Estructura de respuesta correcta")

### 🧪 MICRO-RETO 1: Test de endpoint con error

Crea un test que verifique que un endpoint retorna 404 cuando se busca un recurso inexistente.

In [ ]:
# API con endpoint que puede retornar 404
app_404 = FastAPI()

ITEMS_DB = {
    1: {"nombre": "Laptop", "precio": 999.99},
    2: {"nombre": "Mouse", "precio": 25.50}
}

@app_404.get("/items/{item_id}")
def obtener_item(item_id: int):
    item = ITEMS_DB.get(item_id)
    if not item:
        raise HTTPException(status_code=404, detail="Item no encontrado")
    return item

# TODO: Completa el test
client_404 = TestClient(app_404)

# Test 1: Item existente retorna 200
# TODO: Completa aquí

# Test 2: Item inexistente retorna 404
# TODO: Completa aquí

# Test 3: Verificar mensaje de error
# TODO: Completa aquí

---

## 3. FIXTURES Y MOCKING (15 min + micro-reto)

### ¿Qué son las fixtures en pytest?

Las **fixtures** son funciones reutilizables que preparan el entorno para tests.

**Ventajas:**
- Evitar código duplicado
- Setup/teardown automático
- Tests más limpios y legibles

**Ejemplo básico:**
```python
@pytest.fixture
def client():
    return TestClient(app)

def test_endpoint(client):  # pytest inyecta automáticamente
    response = client.get("/")
    assert response.status_code == 200
```

### Fixture de cliente de testing

En notebooks no podemos usar el decorador `@pytest.fixture`, pero podemos simular el concepto con funciones.

In [ ]:
# Simulación de fixture (en archivo .py usarías @pytest.fixture)
def get_test_client():
    """Retorna un cliente de testing configurado"""
    app = FastAPI()
    
    @app.get("/usuarios")
    def listar_usuarios():
        return [{"id": 1, "nombre": "Ana"}, {"id": 2, "nombre": "Juan"}]
    
    return TestClient(app)

# Uso de la fixture
def test_listar_usuarios():
    client = get_test_client()
    response = client.get("/usuarios")
    
    assert response.status_code == 200
    usuarios = response.json()
    assert len(usuarios) == 2
    assert usuarios[0]["nombre"] == "Ana"

test_listar_usuarios()
print("✅ Test con fixture pasado")

### app.dependency_overrides: Mockear dependencias

En tests, a menudo queremos **mockear** (simular) dependencias como autenticación, bases de datos, etc.

FastAPI tiene `app.dependency_overrides` para reemplazar dependencias en tests.

**Caso de uso típico:** Mockear autenticación para no necesitar tokens reales.

In [ ]:
# API con autenticación
app_auth = FastAPI()

class User(BaseModel):
    username: str
    email: str

# Dependencia real (simula validar token)
def get_current_user() -> User:
    # En producción: validaría JWT, buscaría en BD, etc.
    raise HTTPException(status_code=401, detail="No autenticado")

CurrentUser = Annotated[User, Depends(get_current_user)]

@app_auth.get("/perfil")
def obtener_perfil(current_user: CurrentUser):
    return current_user

# SIN MOCK: Endpoint retorna 401
client_auth = TestClient(app_auth)
response = client_auth.get("/perfil")
assert response.status_code == 401
print("✅ Sin mock: Retorna 401 como esperado")

In [ ]:
# CON MOCK: Reemplazar dependencia
def mock_get_current_user() -> User:
    """Usuario mockeado para tests"""
    return User(username="testuser", email="test@example.com")

# Reemplazar dependencia SOLO en tests
app_auth.dependency_overrides[get_current_user] = mock_get_current_user

# Ahora el endpoint retorna 200 con usuario mockeado
response = client_auth.get("/perfil")
assert response.status_code == 200
assert response.json()["username"] == "testuser"
print("✅ Con mock: Retorna 200 con usuario mockeado")

# IMPORTANTE: Limpiar después del test
app_auth.dependency_overrides = {}
print("✅ Override limpiado")

### Patrón completo: Test con fixture + mock

In [ ]:
def get_test_client_with_mock():
    """Fixture que retorna cliente con autenticación mockeada"""
    app = FastAPI()
    
    # Dependencia real
    def get_user():
        raise HTTPException(status_code=401)
    
    # Mock
    def mock_user():
        return {"id": 1, "username": "testuser"}
    
    @app.get("/protected")
    def protected_route(user=Depends(get_user)):
        return {"mensaje": f"Hola {user['username']}"}
    
    # Reemplazar dependencia
    app.dependency_overrides[get_user] = mock_user
    
    return TestClient(app)

# Test
client = get_test_client_with_mock()
response = client.get("/protected")
assert response.status_code == 200
assert "testuser" in response.json()["mensaje"]
print("✅ Test con fixture + mock completo")

### 🧪 MICRO-RETO 2: Mockear base de datos

Crea un test que mockee una función de "buscar en BD" para retornar datos fake.

In [ ]:
app_db = FastAPI()

# Dependencia que simula consulta a BD
def get_db_connection():
    # En producción: retornaría conexión real a PostgreSQL, etc.
    raise Exception("BD no disponible en tests")

@app_db.get("/productos")
def listar_productos(db=Depends(get_db_connection)):
    # En producción: db.query(...)
    return db

# TODO: Crea una función mock_db que retorne una lista fake de productos
def mock_db():
    # TODO: Retorna [{"id": 1, "nombre": "Laptop"}, {"id": 2, "nombre": "Mouse"}]
    pass

# TODO: Reemplaza la dependencia con app_db.dependency_overrides

# TODO: Haz un test GET /productos y verifica que retorna la lista fake

# TODO: Limpia el override

---

# 🤖 BLOQUE 2: INTEGRACIÓN CON IA GENERATIVA (90 minutos)

## 1. INTEGRACIÓN CON APIS DE IA (10 min + micro-reto)

### Llamada a OpenAI/Anthropic

La librería `openai` (v1+) es compatible con múltiples proveedores:

- **OpenAI** (GPT-4, GPT-3.5)
- **Anthropic** (Claude) → Compatible con OpenAI SDK
- **Otros** (con base_url personalizada)

### Configuración de API keys

**⚠️ IMPORTANTE:** NUNCA hardcodear API keys en el código.

In [ ]:
from openai import OpenAI

# ✅ FORMA CORRECTA: Variable de entorno
# En terminal: export OPENAI_API_KEY="tu-clave"
# O usar archivo .env con python-dotenv

# Configuración del cliente
# Nota: En producción, estas variables vienen del entorno
# Para este notebook educativo, las definimos como None
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("⚠️ OPENAI_API_KEY no configurada (normal en notebook educativo)")
    print("   En producción: export OPENAI_API_KEY='tu-clave'")
else:
    client = OpenAI(api_key=OPENAI_API_KEY)
    print("✅ Cliente OpenAI configurado")

### Estructura básica de request

Todas las APIs de LLM siguen un patrón similar:

```python
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "Eres un asistente útil"},
        {"role": "user", "content": "¿Qué es FastAPI?"}
    ],
    max_tokens=100,
    temperature=0.7
)

texto = response.choices[0].message.content
```

**Parámetros clave:**
- `model`: Modelo a usar (gpt-4, gpt-3.5-turbo, etc.)
- `messages`: Lista de mensajes (system, user, assistant)
- `max_tokens`: Límite de tokens en respuesta
- `temperature`: Creatividad (0 = determinista, 1 = creativo)

### Ejemplo simulado (sin API key real)

Para propósitos educativos, simularemos la estructura de respuesta:

In [ ]:
# Simulación de respuesta de OpenAI para fines educativos
class MockChoice:
    def __init__(self, content):
        self.message = type('obj', (object,), {'content': content})

class MockResponse:
    def __init__(self, content, tokens_used):
        self.choices = [MockChoice(content)]
        self.usage = type('obj', (object,), {
            'prompt_tokens': 10,
            'completion_tokens': tokens_used,
            'total_tokens': 10 + tokens_used
        })

def mock_openai_call(messages, max_tokens=100):
    """Simula llamada a OpenAI"""
    user_msg = messages[-1]["content"]
    response_text = f"Esta es una respuesta simulada a: '{user_msg}'"
    return MockResponse(response_text, tokens_used=15)

# Ejemplo de uso
response = mock_openai_call([
    {"role": "system", "content": "Eres un asistente"},
    {"role": "user", "content": "¿Qué es FastAPI?"}
])

print("Respuesta:", response.choices[0].message.content)
print("Tokens usados:", response.usage.total_tokens)

### Integración en endpoint de FastAPI

In [ ]:
app_ia = FastAPI()

class CompletionRequest(BaseModel):
    prompt: str
    max_tokens: int = 100

class CompletionResponse(BaseModel):
    respuesta: str
    tokens_usados: int

@app_ia.post("/completar", response_model=CompletionResponse)
async def completar_texto(request: CompletionRequest):
    """Endpoint que llama a API de IA"""
    
    # Simular llamada a OpenAI
    response = mock_openai_call(
        messages=[{"role": "user", "content": request.prompt}],
        max_tokens=request.max_tokens
    )
    
    return CompletionResponse(
        respuesta=response.choices[0].message.content,
        tokens_usados=response.usage.total_tokens
    )

# Test
client_ia = TestClient(app_ia)
response = client_ia.post("/completar", json={
    "prompt": "Explica qué es FastAPI",
    "max_tokens": 50
})

assert response.status_code == 200
data = response.json()
print("✅ Endpoint de IA funcionando")
print(f"Respuesta: {data['respuesta'][:50]}...")
print(f"Tokens: {data['tokens_usados']}")

### 🧪 MICRO-RETO 3: Endpoint con prompt personalizado

Crea un endpoint `/traducir` que reciba texto y lo "traduzca" usando un system prompt específico.

In [ ]:
# TODO: Define modelo TraduccionRequest con campos: texto, idioma_destino

# TODO: Crea endpoint POST /traducir que:
# 1. Use system prompt: "Eres un traductor profesional"
# 2. User prompt: f"Traduce al {idioma}: {texto}"
# 3. Llame a mock_openai_call
# 4. Retorne la "traducción"

# TODO: Test: Solicitar traducir "Hello" a "español"

---

## 2. CONTROL DE TOKENS Y COSTES (10 min + micro-reto)

### Token limits

Los LLMs tienen límites de tokens:

- **gpt-3.5-turbo**: 4,096 tokens (input + output)
- **gpt-4**: 8,192 tokens
- **gpt-4-32k**: 32,768 tokens

**1 token ≈ 0.75 palabras en inglés** (más en otros idiomas)

### Estimación de costes

**Precios aproximados (GPT-3.5-turbo):**
- Input: $0.50 / 1M tokens
- Output: $1.50 / 1M tokens

**Ejemplo:**
- Request con 100 tokens input + 200 tokens output
- Coste: (100 × $0.50 + 200 × $1.50) / 1,000,000 = $0.00035

In [ ]:
# Función para estimar costes
def estimar_coste(prompt_tokens: int, completion_tokens: int, modelo="gpt-3.5-turbo"):
    """Estima el coste de una llamada a OpenAI"""
    
    # Precios por millón de tokens (actualizar según pricing oficial)
    PRECIOS = {
        "gpt-3.5-turbo": {"input": 0.50, "output": 1.50},
        "gpt-4": {"input": 30.00, "output": 60.00},
    }
    
    precios = PRECIOS.get(modelo, PRECIOS["gpt-3.5-turbo"])
    
    coste_input = (prompt_tokens * precios["input"]) / 1_000_000
    coste_output = (completion_tokens * precios["output"]) / 1_000_000
    
    return coste_input + coste_output

# Ejemplo
coste = estimar_coste(prompt_tokens=100, completion_tokens=200)
print(f"Coste estimado: ${coste:.6f}")
print(f"Coste por 1000 requests: ${coste * 1000:.2f}")

### Rate limiting básico

Para controlar costes y evitar abusos, implementamos rate limiting.

In [ ]:
from collections import defaultdict
from datetime import datetime, timedelta

# Almacén simple de rate limiting (en producción: usar Redis)
rate_limit_store = defaultdict(list)

def check_rate_limit(user_id: str, max_requests: int = 10, window_minutes: int = 1) -> bool:
    """Verifica si el usuario excedió el límite de requests"""
    now = datetime.now()
    window_start = now - timedelta(minutes=window_minutes)
    
    # Filtrar requests dentro de la ventana temporal
    recent_requests = [
        req_time for req_time in rate_limit_store[user_id]
        if req_time > window_start
    ]
    
    rate_limit_store[user_id] = recent_requests
    
    # Verificar límite
    if len(recent_requests) >= max_requests:
        return False
    
    # Registrar nuevo request
    rate_limit_store[user_id].append(now)
    return True

# Ejemplo de uso
for i in range(12):
    if check_rate_limit("user123", max_requests=10):
        print(f"Request {i+1}: ✅ Permitido")
    else:
        print(f"Request {i+1}: ❌ Rate limit excedido")

### Integración en endpoint con rate limit

In [ ]:
app_rate = FastAPI()

@app_rate.post("/completar-limitado")
async def completar_con_rate_limit(request: CompletionRequest, user_id: str = "default"):
    """Endpoint con rate limiting"""
    
    # Verificar rate limit
    if not check_rate_limit(user_id, max_requests=5, window_minutes=1):
        raise HTTPException(
            status_code=429,
            detail="Rate limit excedido. Intenta en 1 minuto."
        )
    
    # Llamar a IA
    response = mock_openai_call(
        messages=[{"role": "user", "content": request.prompt}],
        max_tokens=request.max_tokens
    )
    
    return {
        "respuesta": response.choices[0].message.content,
        "tokens_usados": response.usage.total_tokens
    }

# Test: Exceder rate limit
client_rate = TestClient(app_rate)
rate_limit_store.clear()  # Limpiar contador

for i in range(7):
    response = client_rate.post("/completar-limitado?user_id=test", json={
        "prompt": f"Request {i+1}"
    })
    if response.status_code == 200:
        print(f"Request {i+1}: ✅ 200")
    elif response.status_code == 429:
        print(f"Request {i+1}: ⚠️ 429 Rate limit")
        break

### 🧪 MICRO-RETO 4: Control de max_tokens

Modifica el endpoint para rechazar requests con `max_tokens > 500` (código 400).

In [ ]:
# TODO: Crea endpoint /completar-seguro que:
# 1. Valide que max_tokens <= 500
# 2. Si excede, retorne HTTPException 400
# 3. Si no, llame a mock_openai_call

# TODO: Test 1: max_tokens=100 → 200 OK
# TODO: Test 2: max_tokens=1000 → 400 Bad Request

---

## 3. MANEJO DE ERRORES (10 min + micro-reto)

### Timeouts largos

Las llamadas a APIs de IA pueden tardar varios segundos. Debemos:

1. **Configurar timeouts** para evitar requests colgados
2. **Informar al usuario** si la operación toma mucho tiempo

In [ ]:
import asyncio

async def llamada_ia_con_timeout(prompt: str, timeout_segundos: int = 30):
    """Llamada a IA con timeout"""
    try:
        # Simular llamada que tarda
        async def llamada_lenta():
            await asyncio.sleep(2)  # Simula latencia
            return mock_openai_call([{"role": "user", "content": prompt}])
        
        # Ejecutar con timeout
        response = await asyncio.wait_for(llamada_lenta(), timeout=timeout_segundos)
        return response
    
    except asyncio.TimeoutError:
        raise HTTPException(
            status_code=504,
            detail=f"Timeout: La IA no respondió en {timeout_segundos}s"
        )

# Test
response = await llamada_ia_con_timeout("Test", timeout_segundos=5)
print("✅ Llamada con timeout exitosa")
print(f"Respuesta: {response.choices[0].message.content[:50]}...")

### Retry logic con backoff exponencial

Si la API falla temporalmente, podemos reintentar con esperas crecientes.

In [ ]:
async def llamada_ia_con_retry(prompt: str, max_intentos: int = 3):
    """Llamada con reintentos y backoff exponencial"""
    
    for intento in range(max_intentos):
        try:
            # Simular que falla las primeras 2 veces
            if intento < 2:
                raise Exception("Error temporal de API")
            
            response = mock_openai_call([{"role": "user", "content": prompt}])
            return response
        
        except Exception as e:
            espera = 2 ** intento  # Backoff exponencial: 1s, 2s, 4s...
            print(f"Intento {intento + 1} falló. Reintentando en {espera}s...")
            
            if intento < max_intentos - 1:
                await asyncio.sleep(espera)
            else:
                raise HTTPException(
                    status_code=503,
                    detail="API de IA no disponible tras múltiples intentos"
                )

# Test
response = await llamada_ia_con_retry("Test retry")
print("✅ Retry exitoso")
print(f"Respuesta: {response.choices[0].message.content[:50]}...")

### Logging de uso

Es crítico registrar cada llamada a IA para monitoreo de costes y debugging.

In [ ]:
import logging

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class IALogger:
    """Logger especializado para llamadas a IA"""
    
    @staticmethod
    def log_request(user_id: str, prompt: str, max_tokens: int):
        logger.info(f"[IA REQUEST] user={user_id} prompt_len={len(prompt)} max_tokens={max_tokens}")
    
    @staticmethod
    def log_response(user_id: str, tokens_used: int, coste: float, duracion_ms: int):
        logger.info(
            f"[IA RESPONSE] user={user_id} tokens={tokens_used} "
            f"coste=${coste:.6f} duracion={duracion_ms}ms"
        )
    
    @staticmethod
    def log_error(user_id: str, error: str):
        logger.error(f"[IA ERROR] user={user_id} error={error}")

# Ejemplo de uso
start_time = time.time()

IALogger.log_request("user123", "¿Qué es FastAPI?", max_tokens=100)
response = mock_openai_call([{"role": "user", "content": "Test"}])

duracion_ms = int((time.time() - start_time) * 1000)
coste = estimar_coste(10, 15)

IALogger.log_response("user123", tokens_used=25, coste=coste, duracion_ms=duracion_ms)
print("✅ Logging configurado")

### Endpoint completo con manejo de errores

In [ ]:
app_completo = FastAPI()

@app_completo.post("/ia/completar")
async def endpoint_ia_completo(
    request: CompletionRequest,
    user_id: str = "default"
):
    """Endpoint de IA con todas las mejores prácticas"""
    
    start_time = time.time()
    
    try:
        # 1. Validar max_tokens
        if request.max_tokens > 500:
            raise HTTPException(status_code=400, detail="max_tokens excede límite de 500")
        
        # 2. Rate limiting
        if not check_rate_limit(user_id, max_requests=5):
            raise HTTPException(status_code=429, detail="Rate limit excedido")
        
        # 3. Log request
        IALogger.log_request(user_id, request.prompt, request.max_tokens)
        
        # 4. Llamar a IA con timeout y retry
        response = await llamada_ia_con_retry(request.prompt, max_intentos=3)
        
        # 5. Calcular métricas
        tokens_used = response.usage.total_tokens
        coste = estimar_coste(response.usage.prompt_tokens, response.usage.completion_tokens)
        duracion_ms = int((time.time() - start_time) * 1000)
        
        # 6. Log response
        IALogger.log_response(user_id, tokens_used, coste, duracion_ms)
        
        # 7. Retornar respuesta
        return {
            "respuesta": response.choices[0].message.content,
            "tokens_usados": tokens_used,
            "coste_estimado": coste,
            "duracion_ms": duracion_ms
        }
    
    except HTTPException:
        raise
    except Exception as e:
        IALogger.log_error(user_id, str(e))
        raise HTTPException(status_code=500, detail="Error interno al procesar request")

# Test
client_completo = TestClient(app_completo)
rate_limit_store.clear()

response = client_completo.post("/ia/completar?user_id=test", json={
    "prompt": "Explica FastAPI en 2 líneas",
    "max_tokens": 50
})

assert response.status_code == 200
data = response.json()
print("\n✅ Endpoint completo funcionando")
print(f"Respuesta: {data['respuesta'][:60]}...")
print(f"Tokens: {data['tokens_usados']}")
print(f"Coste: ${data['coste_estimado']:.6f}")
print(f"Duración: {data['duracion_ms']}ms")

### 🧪 MICRO-RETO 5: Endpoint con logging completo

Añade logs INFO cuando el rate limit es excedido (antes de lanzar el 429).

In [ ]:
# TODO: Modifica check_rate_limit() para que haga:
# logger.info(f"[RATE LIMIT] user={user_id} requests={len(recent_requests)}/{max_requests}")

# TODO: Cuando se exceda, loggea ANTES de retornar False:
# logger.warning(f"[RATE LIMIT EXCEEDED] user={user_id}")

# TODO: Test: Haz 6 requests y verifica que aparece el log de warning